https://raw.githubusercontent.com/Motunrayo244/EDISS-MP/main/MachineLearning/MiniProject2/Sentiment140.tenPercent.sample.tweets.tsv

0=negative
4= positive

In [43]:
!unzip "/content/glove.twitter.27B.zip.1"

In [ ]:
!pip uninstall accelerate transformers -y
!pip install accelerate transformers[torch]

#restart Your session on colab so the new transformer  is applied

In [2]:
! wget https://nlp.stanford.edu/data/glove.twitter.27B.zip
! unzip -q glove.twitter.27B.zip -d ./glovetwitter
# ! wget https://nlp.stanford.edu/data/glove.6B.zip
# ! unzip -q glove.6B.zip -d ./glove6B

In [3]:
import re
import string
from time import perf_counter

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import torch
from keras import utils
from keras.layers import Dense, Flatten, Conv1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from numpy import asarray
from numpy import zeros
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report, \
    confusion_matrix
from sklearn.metrics import roc_curve, RocCurveDisplay, precision_recall_curve, PrecisionRecallDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import pipeline
from wordcloud import WordCloud

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words =set(stopwords.words('english'))

In [5]:
data = pd.read_csv('https://raw.githubusercontent.com/Motunrayo244/EDISS-MP/main/MachineLearning/MiniProject2/Sentiment140.tenPercent.sample.tweets.tsv',sep='\t')

In [6]:
data.head()

In [7]:
data.shape

In [8]:
data['sentiment_label'].value_counts().plot.pie(autopct='%.2f')

In [9]:
text = " ".join(data['tweet_text'])

# Create a WordCloud object
wordcloud = WordCloud(width=1500, height=700, background_color='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Data Preprocessing

In [10]:
def emoji_translator(text):
  emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad',
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed',
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink',
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
  for word in text:
    if word in emojis:
      text = text.replace(word, emojis[word])
  return text


In [11]:
def basic_text_cleaner(text):
  ''' Make text lowercase, remove punctuations and usertags'''

    # Defining regex patterns.
  urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
  sequencePattern   =  r"(.)\1\1+"
  seqReplacePattern = r"\1 "
# '((@ \w*)|(@ \w*))'
  text = text.lower()
  text = re.sub('\@ \w+', '', text)
  text = re.sub('\@\w*', '', text)
  text = re.sub(urlPattern, '', text, flags=re.MULTILINE)
  text = emoji_translator(text)
  text = re.sub(sequencePattern, seqReplacePattern, text)
  text = re.sub('[^a-zA-Z0-9]', ' ', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), ' ',text)
  text = re.sub('\d+', '',text)
  text = re.sub('\w\d\w*', '',text)
  text = re.sub(' +',' ',text)
  text_tokens = word_tokenize(text)
  filtered_text = [w for w in text_tokens if w not in stop_words]
  text = " ".join(filtered_text)

  return text

In [12]:
data['cleaned_text'] = data['tweet_text'].apply(basic_text_cleaner)

In [13]:
text = " ".join(data['cleaned_text'])

# Create a WordCloud object
wordcloud = WordCloud(width=1500, height=700, background_color='white').generate(text)

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [14]:
data.head()

### Machine Learning Models
In this section,
•	Random Forest classifier
•	Linear support Vector Machine
•	Multinomial Naïve Bayes
•	Logistic Regression

will be trained on the dataset and compared based on their Accuracy, F1_score, precision.



In [15]:
data.shape

In [16]:
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(data.cleaned_text)
vectorized_data = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
vectorized_data.head()

In [17]:
X = vectorized_data.copy()
y = data.copy().sentiment_label.apply(lambda x: 1 if x ==4 else x )

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42, shuffle=True, stratify=y )

In [19]:
def evaluate_models(clf,X_train, y_train, X_test, y_test):
  result = {}

  start = perf_counter ()
  train_pred = clf.predict(X_train)
  end = perf_counter()
  result['train_pred_time'] = end - start

  start = perf_counter ()
  test_pred = clf.predict(X_test)
  end = perf_counter()
  result['test_pred_time'] = end - start


  result['train_accuracy'] =  accuracy_score(y_true=y_train, y_pred= train_pred)
  result['train_f1'] =  f1_score(y_true=y_train, y_pred= train_pred)
  result['train_precision'] =  precision_score(y_true=y_train, y_pred= train_pred)
  result['train_recall'] =  precision_score(y_true=y_train, y_pred= train_pred)

  result['test_accuracy'] = accuracy_score(y_true=y_test, y_pred= test_pred)
  result['test_f1'] = f1_score(y_true=y_test, y_pred= test_pred)
  result['test_precision'] = precision_score (y_true=y_test, y_pred= test_pred)
  result['test_recall'] = recall_score(y_true=y_test, y_pred= test_pred)

  return result




In [20]:
# cv = KFold(n_splits=3, random_state=42,shuffle=True)
clf_RF = RandomForestClassifier( n_estimators= 50,  random_state=42)
clf_SVC = LinearSVC()
clf_NB = MultinomialNB()
clf_LR = LogisticRegression(C=1e9, solver='lbfgs', max_iter=1000,random_state=42)

results = {}
models = [clf_RF,clf_SVC,clf_NB,clf_LR]
for clf in models:
  clf_name =  type(clf).__name__
  clf_result = {}
  start = perf_counter()
  clf.fit(X_train, y_train)
  end = perf_counter()
  clf_result['training_time'] = end - start
  clf_result.update(evaluate_models(clf,X_train[0:320000], y_train[0:320000], X_test, y_test))

  results[clf_name] = clf_result


In [21]:
df_results = pd.DataFrame.from_dict(results)
df_results = df_results.transpose()

In [22]:
df_results

In [23]:
plt.figure(figsize=(8, 5))
width = 0.2
x = np.arange(4)
# plot data in grouped manner of bar type
plt.bar(x-0.2, df_results['train_accuracy'],width, color='cyan')
plt.bar(x,df_results['test_accuracy'] , width, color='orange')
plt.bar(x+0.2, df_results['train_f1'], width, color='green')
plt.bar(x+0.4, df_results['test_f1'], width, color='blue')

plt.xticks(x, list(df_results.index))
plt.xlabel("Classifier")
plt.ylabel("Scores")
plt.legend(["train_accuracy", "test_accuracy", "train_f1", "test_f1"])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [24]:
X_train.shape

In [25]:
def evaluate_model(model,X_test,y_test):
  pred = model.predict(X_test)

  accuracy = accuracy_score(y_true=y_test, y_pred= pred)
  f1 = f1_score(y_true=y_test, y_pred= pred)
  precision = precision_score (y_true=y_test, y_pred= pred)
  recall = recall_score(y_true=y_test, y_pred= pred)
  cm = confusion_matrix(y_true=y_test, y_pred=pred)

  total = np.sum(cm)

  cm = cm/total

  cm_display = ConfusionMatrixDisplay(cm).plot()
  print(f"{type(model).__name__}\n{classification_report(y_test, pred)}")

  return {'f1':f1,'acc':accuracy,'precision':precision,'recall':recall}

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,random_state=42, shuffle=True, stratify=y )
LogisticRegression(C=1e9, solver='lbfgs', max_iter=1000,random_state=42)
clf.fit(X_train, y_train)
evaluate_model(model=clf, X_test= X_test, y_test=y_test)





In [27]:
def plot_roc_pr(model, X_test, y_test):
    # Determine scores based on the availability of predict_proba or decision_function
    if hasattr(model, 'predict_proba'):
        y_score = model.predict_proba(X_test)[:, 1]  # Probability estimates for the positive class
    elif hasattr(model, 'decision_function'):
        y_score = model.decision_function(X_test)
    else:
        raise ValueError("Model does not have a predict_proba or a decision_function method.")

    # Calculate ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_score)
    roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr)

    # Calculate Precision-Recall curve
    prec, recall, _ = precision_recall_curve(y_test, y_score)
    pr_display = PrecisionRecallDisplay(precision=prec, recall=recall)

    # Plot both ROC and Precision-Recall curves
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    roc_display.plot(ax=ax1)
    pr_display.plot(ax=ax2)

    ax1.set_title('ROC Curve')
    ax2.set_title('Precision-Recall Curve')
    plt.show()

In [28]:
evaluate_model(model=clf, X_test= X_test, y_test=y_test)
yclf_score = clf.predict_proba(X_test)[:, 1]

# Calculate ROC curve
fpr_clf, tpr_clf, _ = roc_curve(y_test, yclf_score)
clf_roc_display = RocCurveDisplay(fpr=fpr_clf, tpr=tpr_clf)


# Calculate Precision-Recall curve
prec_clf, recall_clf, _ = precision_recall_curve(y_test, yclf_score)
clf_pr_display = PrecisionRecallDisplay(precision=prec_clf, recall=recall_clf)

# Plot both ROC and Precision-Recall curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
clf_roc_display.plot(ax=ax1)

clf_pr_display.plot(ax=ax2)


ax1.set_title('ROC Curve')
ax2.set_title('Precision-Recall Curve')
plt.legend(['XGBClassifier', 'GradientBoostingClassifier'])
plt.show()

### Deep Learning Models

In [29]:
X = []
X = list(data['cleaned_text'])
y = data.copy().sentiment_label.apply(lambda x: 1 if x ==4 else x )

In [30]:
X[4]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.8,random_state=42, shuffle=True, stratify=y )

In [32]:
X = []
X = list(data['cleaned_text'])
y = data.copy().sentiment_label.apply(lambda x: 1 if x ==4 else x )
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1,random_state=42, shuffle=True, stratify=y )

In [33]:
tokenizer = Tokenizer(num_words=800000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [34]:
X_train[4]

In [35]:
VOCAB_SIZE = len(tokenizer.word_index)+1
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [45]:
embeddings_dictionary = dict()
# glove_file = open('glove6B/glove.6B.100d.txt', encoding="utf-8")
glove_file = open('glove.twitter.27B.100d.txt', encoding="utf-8")
for line in glove_file:
  records = line.split()
  word = records[0]
  vector_dimensions = asarray(records[1:], dtype ='float32')
  embeddings_dictionary[word] =vector_dimensions
glove_file.close()


In [46]:
embedding_matrix = zeros((VOCAB_SIZE, 100))
for word, index in tokenizer.word_index.items():
  embedding_vector =embeddings_dictionary.get(word)
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector

In [47]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE,100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(Dense(units=16, activation="relu",
                ))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [48]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

In [49]:
utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [50]:
history = model.fit(X_train, y_train, batch_size=128, epochs=11, verbose=1, validation_split=0.2)

In [51]:
score = model.evaluate(X_test, y_test, verbose=1)

In [52]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [53]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

CNN

In [54]:
model = Sequential()

embedding_layer = Embedding(VOCAB_SIZE, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(Dense(units=16, activation="relu", input_shape=(
100,)))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='ADAM', loss='binary_crossentropy', metrics=['acc'])

In [55]:
print(model.summary())

In [56]:
utils.plot_model(model, to_file='CNN_model_plot.png', show_shapes=True, show_layer_names=True)

In [57]:
history = model.fit(X_train, y_train, batch_size=128, epochs=8, verbose=1, validation_split=0.2)

score = model.evaluate(X_test, y_test, verbose=1)

In [58]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

ROC and Precision recall curve of Logistic Regression

In [59]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

#### Transfer Learning
Disclaimer: Run this part if you have access to a GPU. Also ensure that you restarted the colab session after reinstalling transformer. If you do not restart the colab, a cell in the code will fail with an error requesing for Accelerator.

Transfer learning is a technique that enhances the performance of learners in specific domains by leveraging knowledge from different, yet related, source domains. This approach decreases the reliance on extensive data from the target domain for developing learners.

DistilBERT is a streamlined version of the BERT model (a transformer model), designed to be smaller and more efficient. It was trained on the same data set using a method of self-supervision, with the original BERT model serving as its guide.

We will be using the distilbert-base-uncased model from hugging face

In [60]:
model_name="distilbert-base-uncased"
model =AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [61]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [62]:
X = []
X = list(data['tweet_text'])
y = data.copy().sentiment_label.apply(lambda x: 1 if x ==4 else x )
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42, shuffle=True, stratify=y )
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train, test_size=0.25,random_state=42, shuffle=True, stratify=y_train )

In [63]:
from torch.utils.data import Dataset
'''create a class for te dataset'''

class TwitterDataset(Dataset):
  def __init__ (self, encoding,labels):
    self.encodings = encoding
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)





In [64]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

train_encoding = tokenizer(X_train, truncation= True, padding=True)
val_encoding = tokenizer(X_val, truncation= True, padding=True)
test_encoding = tokenizer(X_test, truncation= True, padding=True)

In [65]:
train_dataset = TwitterDataset(train_encoding, list(y_train))
val_dataset = TwitterDataset(val_encoding, list(y_val))
test_dataset = TwitterDataset(test_encoding, list(y_test))

In [66]:
from torch.utils.data import DataLoader
from transformers import AdamW

# DataLoader setup
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained(model_name)
model.to(device)
optim = AdamW(model.parameters(), lr=2e-5)
# model.train()

In [67]:
def evaluate(model, val_loader, device):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(device)
            outputs = model(**inputs, labels=labels)
            val_loss += outputs.loss.item()
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            predictions.extend(preds.tolist())
            true_labels.extend(labels.tolist())

    accuracy = correct / total
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return val_loss / len(val_loader), accuracy, precision, recall,f1


In [68]:
# Training loop with evaluation and checkpointing
best_val_accuracy = 0.0
best_f1 =0.0
model_save_path = 'best_model_state.bin'
num_train_epochs = 2

for epoch in range(num_train_epochs):
    model.train()
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

    val_loss, val_accuracy, val_precision, val_recall, val_f1,  = evaluate(model, val_loader, device)
    print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val precision: {val_precision:.4f},Val recall: {val_recall:.4f}, Val F1 Score: {val_f1:.4f}")

    # Checkpointing
    if val_accuracy > best_val_accuracy and val_f1 > best_f1:
        best_val_accuracy = val_accuracy
        best_f1 = val_f1
        torch.save(model.state_dict(), model_save_path)
        print(f"New best model saved with accuracy: {val_accuracy:.4f} and f1 {val_f1:.4f}")


In [69]:
# Load best model for prediction
model.load_state_dict(torch.load(model_save_path))
model.eval()

# Prediction on test set
test_predictions, test_labels = [], []
with torch.no_grad():
    for batch in test_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        test_predictions.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

In [70]:
def evaluate_bert_model(test_labels, test_predictions):
  accuracy = accuracy_score(y_true=test_labels, y_pred= test_predictions)
  f1 = f1_score(y_true=test_labels, y_pred= test_predictions)
  precision = precision_score (y_true=test_labels, y_pred= test_predictions)
  recall = recall_score(y_true=test_labels, y_pred= test_predictions)
  cm = confusion_matrix(y_true=test_labels, y_pred= test_predictions)

  cm_display = ConfusionMatrixDisplay(cm).plot()
  print(f"{type(model).__name__}\n{classification_report(test_labels, test_predictions)}")

  return {'f1':f1,'acc':accuracy,'precision':precision,'recall':recall}

In [71]:
evaluate_bert_model(test_labels, test_predictions)

In [72]:
# Calculate performance metrics
test_accuracy = accuracy_score(test_labels, test_predictions)
test_precision = precision_score (test_labels, test_predictions)
test_recall =recall_score (test_labels, test_predictions)
test_f1 = f1_score(test_labels, test_predictions)

print(f"Test Accuracy: {test_accuracy:.4f}, Test F1 Score: {test_f1:.4f}")